**GENERADOR DE FEEDBACK AMB IA GENERATIVA**

In [ ]:
from src.core import OPENAI_KEY, CANVAS_BASE_URL, CANVAS_TOKEN, DB_FILEPATH
from src.controllers import FeedbackController
from src.services import ServiceManager, OpenAIService, CanvasService, SQLite3Service

In [ ]:
openai_service = OpenAIService(OPENAI_KEY, "gpt-3.5-turbo")
canvas_service = CanvasService(CANVAS_BASE_URL, CANVAS_TOKEN)
sqlite_service = SQLite3Service(DB_FILEPATH)

service_manager = ServiceManager(openai_service, canvas_service, sqlite_service)
feedback_controller = FeedbackController(service_manager)

course_id = "37205"
assignment_id = "434722"
user_id = "300516"

In [ ]:
# feedback_controller.add_data(course_id, assignment_id)

In [ ]:
data = feedback_controller.data

course = next((course for course in data if course.id == course_id), None)
assignment = next((assignment for assignment in course.assignments if assignment.id == assignment_id), None)
rubric = assignment.rubric
submission = next((submission for submission in assignment.submissions if submission.user_id == user_id), None)
assessments = submission.assessments
attachment_content = feedback_controller.get_attachment_content(submission.attachment, ["ex. nom_estudiant"])

data_content = {
    "course": course,
    "assignment": assignment,
    "rubric": rubric,
    "submission": submission,
    "assessments": assessments,
    "attachment_content": attachment_content,
} 

In [ ]:
feedback_controller.clear_prompt(['user'])
feedback_controller.generate_prompt(**data_content)
feedback_controller.generate_response()

In [ ]:
feedback_controller.publish_feedback(course_id, assignment_id, user_id)